In [10]:
import pandas as pd 
from geopy.distance import geodesic
pd.set_option('display.max_colwidth', None)


In [11]:
df = pd.read_csv('train.csv')
#limpandom os dados 
df1 = df.copy()
linhas_selecionadas = (df1["Delivery_person_Age"] != "NaN ")
df1 = df1.loc[linhas_selecionadas, :].copy()
df1["Delivery_person_Age"] = df1["Delivery_person_Age"].astype( int )
df1["Delivery_person_Ratings"] = df1["Delivery_person_Ratings"].astype( float)
df1["Order_Date"] = pd.to_datetime(df1["Order_Date"], format='%d-%m-%Y')
linhas_selecionadas = (df1["multiple_deliveries"] != "NaN ")
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype( int )
#limpando os espaços nas linhas
df1.loc[: , 'ID'] = df1.loc[: , 'ID'].str.strip()
df1.loc[: , 'Delivery_person_ID'] = df1.loc[: , 'Delivery_person_ID'].str.strip()
df1.loc[: , 'Road_traffic_density'] = df1.loc[: , 'Road_traffic_density'].str.strip()
df1.loc[: , 'Type_of_order'] = df1.loc[: , 'Type_of_order'].str.strip()
df1.loc[: , 'Type_of_vehicle'] = df1.loc[: , 'Type_of_vehicle'].str.strip()
df1.loc[: , 'Festival'] = df1.loc[: , 'Festival'].str.strip()
df1.loc[: , 'City'] = df1.loc[: , 'City'].str.strip()
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')
df1 =  df1.loc[df1['City'] != 'NaN' , :]
df1 = df1.loc[df1['Weatherconditions'] != 'conditions NaN' , :]
df1 = df1.loc[df1['Road_traffic_density'] !=  'NaN' , :]
# Remover "(min)" e espaços extras, depois converter para número
df1['Time_taken(min)'] = df1['Time_taken(min)'].str.extract(r'(\d+)').astype(float)
 


In [12]:
#6. Os 10 entregadores mais rápidos por cidade.
#soma de todos os minutos de pedidos
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby([ 'City' , 'Delivery_person_ID']).min().sort_values(['City' , 'Time_taken(min)']).reset_index()
df2_aux1 = df2.loc[df2[ 'City' ] ==  'Metropolitian' , : ].head(10)
df2_aux2 = df2.loc[df2[ 'City' ] ==  'Urban' , : ].head(10)
df2_aux3 = df2.loc[df2[ 'City' ] ==  'Semi-Urban' , : ].head(10)
df2_aux4 =  pd.concat([df2_aux1, df2_aux2, df2_aux3 ]).reset_index(drop=True) 


In [13]:
df1


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24.0,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33.0,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26.0,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21.0,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,11:35:00,...,conditions Windy,High,1,Meal,motorcycle,0,No,Metropolitian,32.0,12
45589,0xd641,AGRRES16DEL01,21,4.6,0.000000,0.000000,0.070000,0.070000,2022-02-16,19:55:00,...,conditions Windy,Jam,0,Buffet,motorcycle,1,No,Metropolitian,36.0,07
45590,0x4f8d,CHENRES08DEL03,30,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,23:50:00,...,conditions Cloudy,Low,1,Drinks,scooter,0,No,Metropolitian,16.0,10
45591,0x5eee,COIMBRES11DEL01,20,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,13:35:00,...,conditions Cloudy,High,0,Snack,motorcycle,1,No,Metropolitian,26.0,10


In [14]:
#1. A quantidade de entregadores únicos.
entregadores_unicos = len(df1.loc[: , 'Delivery_person_ID'].unique())
entregadores_unicos

1320

In [15]:
#A distância média dos resturantes e dos locais de entrega.
#sa = um valor ( distancia media ) 
#p =  calcular a distancia de cada  pedidos aos restaurantes formando uma lista ou uma nova columa
#e = colunas de long e lat dos restaurantes e enedereço de entregas, procurar um comando que calucule isso 
def calcular_distancia(row):
    ponto_restaurante = (row['Restaurant_latitude'], row['Restaurant_longitude'])
    ponto_entrega = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
    return geodesic(ponto_restaurante, ponto_entrega).kilometers
df1['Distancia'] = df1.apply(calcular_distancia, axis=1 )

# calcular a media da coluna distancia 
media_da_distancia = df1.loc[: , 'Distancia'].mean()

cols = ['Delivery_person_ID' ,  'Distancia']

media_da_distancia_por_entregador = df1.groupby('Delivery_person_ID')['Distancia'].mean()

print(media_da_distancia_por_entregador)

Delivery_person_ID
AGRRES010DEL01    470.562636
AGRRES010DEL02      9.130756
AGRRES010DEL03     12.262395
AGRRES01DEL01       9.909364
AGRRES01DEL02      11.147774
                     ...    
VADRES19DEL02      11.018347
VADRES19DEL03       9.196943
VADRES20DEL01       8.514034
VADRES20DEL02      10.955918
VADRES20DEL03       8.807453
Name: Distancia, Length: 1320, dtype: float64


In [16]:
df1.dtypes


ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int64
Festival                               object
City                                   object
Time_taken(min)                       float64
week_of_year                           object
Distancia                         

In [17]:
df1.head(10)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year,Distancia
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,High,2,Snack,motorcycle,0,No,Urban,24.0,11,3.020737
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,Jam,2,Snack,scooter,1,No,Metropolitian,33.0,12,20.143737
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,Low,0,Drinks,motorcycle,1,No,Urban,26.0,11,1.549693
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21.0,14,7.774497
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,High,1,Snack,scooter,1,No,Metropolitian,30.0,12,6.197898
5,0x9bb4,HYDRES09DEL03,22,4.8,17.431668,78.408321,17.461668,78.438321,2022-03-11,21:20:00,...,Jam,0,Buffet,motorcycle,1,No,Urban,26.0,10,4.602205
6,0x95b4,RANCHIRES15DEL01,33,4.7,23.369746,85.339820,23.479746,85.449820,2022-03-04,19:15:00,...,Jam,1,Meal,scooter,1,No,Metropolitian,40.0,09,16.576887
7,0x9eb2,MYSRES15DEL02,35,4.6,12.352058,76.606650,12.482058,76.736650,2022-03-14,17:25:00,...,Medium,2,Meal,motorcycle,1,No,Metropolitian,32.0,11,20.164988
8,0x1102,HYDRES05DEL02,22,4.8,17.433809,78.386744,17.563809,78.516744,2022-03-20,20:55:00,...,Jam,0,Buffet,motorcycle,1,No,Metropolitian,34.0,12,19.940219
9,0xcdcd,DEHRES17DEL01,36,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55:00,...,Jam,2,Snack,motorcycle,3,No,Metropolitian,46.0,06,10.271464


In [18]:
Distancia_maxima = df1[df1['Delivery_person_ID'] == 'AGRRES010DEL01'].groupby('Delivery_person_ID')['Distancia'].value_counts().reset_index()


print(Distancia_maxima)

  Delivery_person_ID    Distancia  count
0     AGRRES010DEL01     5.946101      4
1     AGRRES010DEL01     1.486612      3
2     AGRRES010DEL01    16.349552      3
3     AGRRES010DEL01    10.405070      2
4     AGRRES010DEL01  6019.211231      1


In [19]:
tolerancia = 0.0001  # Ajuste a precisão se necessário

resultado = df1[
    (df1['Distancia'].between(5.946101 - tolerancia, 5.946101 + tolerancia)) | 
    (df1['Distancia'].between(1.486612 - tolerancia, 1.486612 + tolerancia)) |
    (df1['Distancia'].between(16.349552 - tolerancia, 16.349552 + tolerancia)) |
    (df1['Distancia'].between(10.405070 - tolerancia, 10.405070 + tolerancia)) |
    (df1['Distancia'].between(6019.211231 - tolerancia, 6019.211231 + tolerancia))
]

resultado

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year,Distancia
92,0xc042,AGRRES010DEL01,34,4.7,-27.163303,78.057044,27.233303,78.127044,2022-02-16,23:00:00,...,Low,1,Drinks,scooter,0,No,Metropolitian,15.0,07,6019.211231
266,0xd92f,AGRRES03DEL01,35,4.9,27.160832,78.011608,27.230832,78.081608,2022-02-18,21:15:00,...,Jam,0,Buffet,motorcycle,2,No,Metropolitian,53.0,07,10.405170
2114,0xd2bf,AGRRES17DEL01,36,4.8,27.157772,78.047250,27.167772,78.057250,2022-02-11,08:20:00,...,Low,2,Meal,scooter,0,No,Metropolitian,24.0,06,1.486644
4266,0xd581,AGRRES03DEL01,37,5.0,27.160832,78.011608,27.230832,78.081608,2022-02-12,23:30:00,...,Low,1,Buffet,scooter,1,No,Urban,25.0,06,10.405170
4558,0xdebb,AGRRES010DEL01,38,4.7,27.163303,78.057044,27.233303,78.127044,2022-02-18,22:45:00,...,Low,1,Buffet,motorcycle,1,No,Urban,28.0,07,10.405070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43491,0xe0f7,AGRRES20DEL01,24,5.0,27.158822,78.045359,27.168822,78.055359,2022-02-13,08:25:00,...,Low,1,Buffet,scooter,1,No,Urban,12.0,07,1.486638
44293,0xc612,AGRRES11DEL01,29,4.7,27.163303,78.057044,27.203303,78.097044,2022-02-13,14:30:00,...,High,2,Snack,motorcycle,1,No,Metropolitian,29.0,07,5.946101
44378,0xbfce,AGRRES08DEL01,32,4.6,27.160934,78.044095,27.200934,78.084095,2022-02-17,15:40:00,...,Medium,1,Drinks,motorcycle,0,No,Metropolitian,26.0,07,5.946155
44864,0xcfd3,AGRRES01DEL01,23,4.6,27.161661,78.011544,27.201661,78.051544,2022-02-17,15:50:00,...,Medium,1,Snack,motorcycle,1,No,Metropolitian,23.0,07,5.946139


3. O tempo médio e o desvio padrão de entrega por cidade.


In [20]:
cols = [  'City', 'Time_taken(min)', 'Type_of_order']
tempo_medio_por_entrega_e_cidade =  df1.loc[: , cols].groupby(['City']).agg( {'Time_taken(min)' : ['mean', 'std']} ).reset_index()
tempo_medio_por_entrega_e_cidade

City Time_taken(min)          
                            mean       std
0  Metropolitian       27.375731  9.164721
1     Semi-Urban       49.710526  2.724992
2          Urban       23.096608  8.894365

4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.


In [21]:
cols = [  'City', 'Time_taken(min)', 'Type_of_order' , 'Road_traffic_density']
tempo_medio_por_entrega_e_cidade_e_pedido = df1.loc[: , cols].groupby(['City' ,'Type_of_order']).agg( {'Time_taken(min)' : ['mean', 'std']} )

tempo_medio_por_entrega_e_cidade_e_pedido


Time_taken(min)          
                                       mean       std
City          Type_of_order                          
Metropolitian Buffet              27.252377  9.180606
              Drinks              27.275139  9.071588
              Meal                27.554270  9.249682
              Snack               27.415381  9.152578
Semi-Urban    Buffet              49.707317  2.731702
              Drinks              49.625000  2.459347
              Meal                50.300000  3.041665
              Snack               49.408163  2.707385
Urban         Buffet              23.449257  9.087620
              Drinks              23.157568  8.978597
              Meal                22.948839  8.803883
              Snack               22.831592  8.695209

5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.


In [22]:
tempo_medio_por_entrega_e_cidade_e_trafefo =  df1.loc[ : , cols ].groupby(['City' , 'Road_traffic_density' ]).agg( {'Time_taken(min)' : ['mean', 'std']} )
tempo_medio_por_entrega_e_cidade_e_trafefo

Time_taken(min)           
                                              mean        std
City          Road_traffic_density                           
Metropolitian High                       28.103888   7.935947
              Jam                        31.937908   9.509890
              Low                        22.193973   6.827415
              Medium                     27.694076   8.332809
Semi-Urban    High                       50.125000   2.629956
              Jam                        49.841270   2.717095
              Medium                     47.400000   2.011080
Urban         High                       24.206081   8.535574
              Jam                        27.905200  10.125553
              Low                        19.336444   6.351539
              Medium                     23.764470   8.440659

6. O tempo médio de entrega durantes os Festivais.


In [92]:
tempo_medio_durante_festival =  df1.loc[df1['Festival'] ==  'Yes' , 'Time_taken(min)'].mean()
tempo_medio_durante_festival

np.float64(45.51860744297719)

In [100]:
import folium
from folium import plugins

# Criar o mapa centrado no restaurante
map_center = [df1['Restaurant_latitude'].iloc[0], df1['Restaurant_longitude'].iloc[0]]
mapa = folium.Map(location=map_center, zoom_start=14)

# Plotando as linhas entre o restaurante e os endereços de entrega
for _, row in df1.iterrows():
    folium.Marker(
        location=[row['Restaurant_latitude'], row['Restaurant_longitude']],
        popup='Restaurante',
        icon=folium.Icon(color='blue')
    ).add_to(mapa)

    folium.Marker(
        location=[row['Delivery_location_latitude'], row['Delivery_location_longitude']],
        popup='Entrega',
        icon=folium.Icon(color='red')
    ).add_to(mapa)

    folium.PolyLine(
        locations=[ 
            [row['Restaurant_latitude'], row['Restaurant_longitude']], 
            [row['Delivery_location_latitude'], row['Delivery_location_longitude']]
        ],
        color='green',
        weight=2

_IncompleteInputError: incomplete input (1485085920.py, line 28)

In [95]:
df1.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int64
Festival                               object
City                                   object
Time_taken(min)                       float64
week_of_year                           object
dtype: object

In [25]:
import folium

# Criar o mapa centralizado no primeiro restaurante
map_center = [df1['Restaurant_latitude'].iloc[0], df1['Restaurant_longitude'].iloc[0]]
mapa = folium.Map(location=map_center, zoom_start=4)
df1 = df1.query('Distancia <= 100')

# Filtrar apenas as 10 primeiras linhas do DataFrame
df_test = df1.sample(100)
# Adicionar marcadores e linhas ao mapa
for _, row in df_test.iterrows():
    # Marcador do restaurante
    folium.Marker(
        location=[row['Restaurant_latitude'], row['Restaurant_longitude']],
        popup='Restaurante',
        icon=folium.Icon(color='blue')
    ).add_to(mapa)

    # Marcador do endereço de entrega
    folium.Marker(
        location=[row['Delivery_location_latitude'], row['Delivery_location_longitude']],
        popup='Entrega',
        icon=folium.Icon(color='lightgreen')
    ).add_to(mapa)

    # Adicionar linha entre restaurante e entrega
    folium.PolyLine(
        locations=[
            [row['Restaurant_latitude'], row['Restaurant_longitude']], 
            [row['Delivery_location_latitude'], row['Delivery_location_longitude']]
        ],
        color='red',
        weight=3
    ).add_to(mapa)

# Salvar o mapa como um arquivo HTML para visualizar no navegador
mapa.save("mapa_entregas_teste.html")

mapa 


In [131]:
df1


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year,Distancia
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,High,2,Snack,motorcycle,0,No,Urban,24.0,11,3.020737
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,Jam,2,Snack,scooter,1,No,Metropolitian,33.0,12,20.143737
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,Low,0,Drinks,motorcycle,1,No,Urban,26.0,11,1.549693
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21.0,14,7.774497
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,High,1,Snack,scooter,1,No,Metropolitian,30.0,12,6.197898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,11:35:00,...,High,1,Meal,motorcycle,0,No,Metropolitian,32.0,12,1.488112
45589,0xd641,AGRRES16DEL01,21,4.6,0.000000,0.000000,0.070000,0.070000,2022-02-16,19:55:00,...,Jam,0,Buffet,motorcycle,1,No,Metropolitian,36.0,07,10.983242
45590,0x4f8d,CHENRES08DEL03,30,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,23:50:00,...,Low,1,Drinks,scooter,0,No,Metropolitian,16.0,10,4.648024
45591,0x5eee,COIMBRES11DEL01,20,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,13:35:00,...,High,0,Snack,motorcycle,1,No,Metropolitian,26.0,10,6.219668


4363